In [110]:
from aocd import get_data, submit
# from aocd.get import current_day

from IPython.display import display
from IPython.core.display import Markdown

import math
import numpy as np
import re
import sys
sys.setrecursionlimit(15000)


In [2]:
DAY = 16
YEAR = 2023

In [3]:
url = f"https://adventofcode.com/{YEAR}/day/{DAY}"
display(Markdown(f"#### See [{YEAR} Day {DAY}]({url})."))

#### See [2023 Day 16](https://adventofcode.com/2023/day/16).

In [166]:
data = get_data(year=YEAR, day=DAY)

In [167]:
data.splitlines()[:5]

['\\..................-..........................-............................................-...-.............|',
 '..........-........//....../...-....../.....|...............................|.............-.....-.............',
 '..\\....\\............./...\\.......\\/.................................................../................-......',
 '|...................................................\\...................|...||-..............|................',
 '.....\\.................\\.........................|-.....|.........-.\\...............-.........|....\\.......\\..']

# Part A

In [216]:
data = r""".|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....
"""

In [541]:
data = np.array([list(l) for l in data.splitlines()])
data

array([['\\', '.', '.', ..., '.', '.', '|'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '\\', ..., '.', '.', '.'],
       ...,
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '\\', ..., '.', '.', '.']], dtype='<U1')

In [544]:
up = np.array((-1, 0))
down = np.array((1, 0))
left = np.array((0, -1))
right = np.array((0, 1))

pieces = {
    '\\': {tuple(left): [down], tuple(down): [left], tuple(up): [right], tuple(right): [up]},
    '/': {tuple(left):[up], tuple(up): [left], tuple(down): [right], tuple(right): [down]},
    '-': {tuple(left): [right], tuple(right): [left], tuple(up): [left, right], tuple(down): [left, right]},
    '|': {tuple(up): [down], tuple(down): [up], tuple(left): [up, down], tuple(right): [up, down]},
}

In [116]:
energized = np.zeros(data.shape)
start = np.array([0,-1])

visited = []

def move(last, current):
    # print(f"current {current}")
    state = tuple(last) + tuple(current)
    if state in visited:
        return
    else:
        visited.append(state)

    if any(current < np.array([0,0])):
        return
    elif any(current >= data.shape):
        return

    energized[tuple(current)] = 1
    dir = current - last
    # print(f"dir: {dir}")
    # print(f"piece {data[tuple(current)]}")
    if data[tuple(current)] == '.':
        move(current, current+dir)
    else:
        # print(f"moves {pieces[data[tuple(current)]][tuple(-dir)]}")
        for d in pieces[data[tuple(current)]][tuple(-dir)]:
            move(current, current+d)


move(start, start+right)

In [117]:
energized

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 0., 0., ..., 0., 1., 1.],
       [1., 0., 0., ..., 0., 1., 1.],
       ...,
       [1., 1., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [118]:
total = np.sum(energized)
total

7067.0

In [115]:
submit(total, year=YEAR, day=DAY)

answer a: None
submitting for part a
coerced float64 value 7067.0 for 2023/16


That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


# Part B

In [156]:
def move(last, current):
    # print(f"current {current}")
    state = tuple(last) + tuple(current)
    if state in visited:
        return
    else:
        visited.append(state)

    if any(current < np.array([0,0])):
        return
    elif any(current >= data.shape):
        return

    energized[tuple(current)] = 1
    dir = current - last
    # print(f"dir: {dir}")
    # print(f"piece {data[tuple(current)]}")
    if data[tuple(current)] == '.':
        move(current, current+dir)
    else:
        # print(f"moves {pieces[data[tuple(current)]][tuple(-dir)]}")
        for d in pieces[data[tuple(current)]][tuple(-dir)]:
            move(current, current+d)




In [157]:
starts = [[np.array([-1,i]), np.array([-1,i]) + down] for i in range(data.shape[1])] + \
    [[np.array([data.shape[0],i]), np.array([data.shape[0],i]) + up] for i in range(data.shape[1])] + \
    [[np.array([i, -1]), np.array([i, -1]) + right] for i in range(data.shape[0])] + \
    [[np.array([i, data.shape[1]]), np.array([i, data.shape[1]]) + left] for i in range(data.shape[1])]
maxe = 0
for p1, p2 in starts:
    energized = np.zeros(data.shape)
    visited = []
    move(p1, p2)
    maxe = max(maxe, np.sum(energized))

In [139]:
maxe

7324.0

In [140]:
submit(maxe, year=YEAR, day=DAY)

answer a: 7067
submitting for part b (part a is already completed)
coerced float64 value 7324.0 for 2023/16


That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 16! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# No globals

In [545]:
def move(visited, last, current):
    # print(f"current {current}")
    state = tuple(last) + tuple(current)
    if state in visited:
        return []
    else:
        visited.append(state)

    if any(current < np.array([0,0])):
        return [tuple(last)]
    elif any(current >= data.shape):
        return [tuple(last)]

    energized[tuple(current)] = 1
    dir = current - last
    # print(f"dir: {dir}")
    # print(f"piece {data[tuple(current)]}")
    path = [tuple(last)]
    if data[tuple(current)] == '.':
        path += move(visited, current, current+dir)
    else:
        # print(f"moves {pieces[data[tuple(current)]][tuple(-dir)]}")
        for d in pieces[data[tuple(current)]][tuple(-dir)]:
            path += move(visited, current, current+d)
    return path

In [546]:
starts = [[np.array([-1,i]), np.array([-1,i]) + down] for i in range(data.shape[1])] + \
    [[np.array([data.shape[0],i]), np.array([data.shape[0],i]) + up] for i in range(data.shape[1])] + \
    [[np.array([i, -1]), np.array([i, -1]) + right] for i in range(data.shape[0])] + \
    [[np.array([i, data.shape[1]]), np.array([i, data.shape[1]]) + left] for i in range(data.shape[1])]
maxe = 0
for p1, p2 in starts:
    energized = np.zeros(data.shape)
    p = move([], p1, p2)
    maxe = max(maxe, len(set(p)) - 1)

In [547]:
maxe

7324

In [140]:
submit(maxe, year=YEAR, day=DAY)

answer a: 7067
submitting for part b (part a is already completed)
coerced float64 value 7324.0 for 2023/16


That's the right answer!  You are one gold star closer to restoring snow operations.You have completed Day 16! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


# Caching

In [548]:
data = get_data(year=YEAR, day=DAY)

In [609]:
tiles,L,R,U,D,V,H=[x.strip()for x in data.splitlines()*2],'L','R','U','D','UD','LR'
dirs,n,m,={R:(1,0),L:(-1,0),U:(0,-1),D:(0,1)},list(range(len(tiles))),list(range(len(tiles[0])))
M={R:{'/':U,'\\':D,'|':V},L:{'/':D,'\\':U,'|':V},U:{'/':R,'\\':L,'-':H},D:{'/':L,'\\':R,'-':H}}
S=[(x,0,D)for x in m]+[(0,y,R)for y in n]+[(x,n[-1],U)for x in m]+[(m[-1],y,L)for y in n]

In [612]:
%%time
# BFS
def traverse(s):
  lp,Q = set(),[s]
  while Q:
    x, y, dir = Q.pop(0)
    if (x, y, dir) not in lp and x in (m) and y in (n):
      lp.add((x, y, dir))
      [Q.append((x+dirs[d][0],y+dirs[d][1],d))for d in M[dir].get(tiles[y][x],dir)]
  return len({(x, y) for x, y, _ in lp})
p = traverse((0,0,R))
pp = max(traverse(s) for s in S)
print(p)
print(pp)

15779
15836
CPU times: user 14.8 s, sys: 107 ms, total: 14.9 s
Wall time: 15 s


In [584]:
%%time
# DFS
lp = []
def traverse(s):
    global lp
    x, y, dir = s
    if s not in lp and x in (m) and y in (n):
        path = [(x,y,dir)]
        for d in M[dir].get(tiles[y][x],dir):
            lp += [s]
            path += traverse((x+dirs[d][0],y+dirs[d][1],d))
    else:
        path = []
    return path
p = len({(x,y) for x,y,_ in traverse((0,0,R))})
e = []
for s in S:
    lp = []
    e.append(len({(x,y) for x,y,_ in traverse(s)}))
pp = max(e)
print(p)
print(pp)

7067
7324
CPU times: user 2min 57s, sys: 622 ms, total: 2min 58s
Wall time: 2min 59s


In [613]:
def deref1l(l):
    result = set([x for x in l if not isinstance(x,list)])
    for ll in [x for x in l if isinstance(x, list)]:
        result |= set([x for x in ll if not isinstance(x,list)])
    return list(result)

In [614]:
%%time
# DFS cached
cache = defaultdict(list)
def traverse(s):
    global lp
    x, y, dir = s
    if s in cache:
        return deref1l([x for x in cache[s] if x != cache[s]])
    path = []
    if s in lp:
        path += [cache[s]]
    elif x in (m) and y in (n):
        path += [s]
        for d in M[dir].get(tiles[y][x],dir):
            lp += [s]
            path += traverse((x+dirs[d][0],y+dirs[d][1],d))
    cache[s] += path
    return path
lp = []
p = len({(x,y) for x,y,_ in deref1l(traverse((0,0,R)))})
e = []
for s in S:
    lp = []
    e.append(len({(x,y) for x,y,_ in deref1l(traverse(s))}))
pp = max(e)
print(p)
print(pp)

KeyboardInterrupt: 